In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformers.cache_utils import  DynamicCache

output_dir = 'model'

tokenizer = AutoTokenizer.from_pretrained(output_dir)

model = AutoModelForCausalLM.from_pretrained(
    output_dir,
    device_map='auto',
    torch_dtype=torch.bfloat16,
    use_cache=True,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    attn_implementation="flash_attention_2",
)

model.generation_config.eos_token_id = tokenizer.eos_token_id
model.generation_config.pad_token_id = tokenizer.pad_token_id

/home/ubuntu/miniconda3/envs/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.42it/s]


In [6]:

past_key_values = DynamicCache()
max_cache_length = past_key_values.get_max_length()

user_inputs = ''

messages = []
while True:

    user_inputs = input('Your input: ')
    if user_inputs == 'exit':
        break

    messages.append({"role": "user", "content": user_inputs})

    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt", return_dict=True).to(model.device)
    input_length = inputs["input_ids"].shape[1]

    outputs = model.generate(**inputs, max_new_tokens=512, temperature=1, top_p = 0.85, past_key_values=past_key_values)
    completion = tokenizer.decode(outputs[0, input_length: ], skip_special_tokens=True)
    print(user_inputs)
    print()
    print(completion)
    print()
    
    messages.append({"role": "assistant", "content": completion})


Hello! Who are you?

I am a cutting knowledge AI assistant.

Tell me about NLP and LLM.

Natural language processing (NLP) is the field of computer science that deals with the interaction between computers and human language. It involves a range of techniques for processing and understanding natural language, including text analysis, speech recognition, machine learning, and natural language generation. Natural language processing is widely used in fields such as healthcare, finance, and customer service, and is becoming increasingly important in the world of artificial intelligence.

Large language models (LLMs) are a subset of machine learning models that use deep learning techniques to generate text in response to prompts. LLMs are trained on vast amounts of text data, and can be used for a wide range of tasks, including generating text from images, answering questions, and translating languages. LLMs are becoming increasingly important in the field of artificial intelligence, and a